In [1]:
!pip install transformers datasets


In [2]:
from datasets import load_dataset

# Load dataset
ds = load_dataset("gretelai/symptom_to_diagnosis")

# Inspect one example
print(ds["train"][0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.jsonl: 0.00B [00:00, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/853 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/212 [00:00<?, ? examples/s]

{'output_text': 'cervical spondylosis', 'input_text': "I've been having a lot of pain in my neck and back. I've also been having trouble with my balance and coordination. I've been coughing a lot and my limbs feel weak."}


In [3]:
# Split the test dataset into validation and test sets
ds["validation"] = ds["test"].select(range(100))

In [4]:
{'input_text': 'fever, cough, sore throat', 'output_text': 'flu'}


{'input_text': 'fever, cough, sore throat', 'output_text': 'flu'}

In [5]:
# Collect all unique diseases
unique_labels = list(set(ds["train"]["output_text"]))
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

def convert_to_classification(example):
    return {
        "text": example["input_text"],
        "label": label2id[example["output_text"]]
    }

ds_classification = ds.map(convert_to_classification)


Map:   0%|          | 0/853 [00:00<?, ? examples/s]

Map:   0%|          | 0/212 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [6]:
{'text': 'fever, cough, sore throat', 'label': 12}


{'text': 'fever, cough, sore throat', 'label': 12}

In [7]:
from transformers import AutoTokenizer

model_name = "emilyalsentzer/Bio_ClinicalBERT"  # or "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

encoded = ds_classification.map(tokenize, batched=True)


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/853 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/212 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [8]:
from transformers import AutoModelForSequenceClassification

num_labels = len(unique_labels)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded["train"],
    eval_dataset=encoded["test"].select(range(100)), # Use the newly created validation dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipython-input-326405666.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.639915,0.890000,0.887464
2,No log,0.377908,0.920000,0.921131
3,No log,0.319895,0.900000,0.896020
4,No log,0.166523,0.960000,0.959673
5,0.287000,0.136571,0.970000,0.970241
6,0.287000,0.138602,0.970000,0.970012
7,0.287000,0.112772,0.980000,0.979911
8,0.287000,0.116739,0.980000,0.979911
9,0.287000,0.107969,0.980000,0.979911
10,0.019100,0.108495,0.980000,0.979911


TrainOutput(global_step=1070, training_loss=0.1438711257181435, metrics={'train_runtime': 106.7605, 'train_samples_per_second': 79.898, 'train_steps_per_second': 10.022, 'total_flos': 245068739313300.0, 'train_loss': 0.1438711257181435, 'epoch': 10.0})

In [21]:
# Evaluate the model on the test dataset
test_results = trainer.evaluate(encoded["test"])

print("Test Results:")
print(test_results)

Test Results:
{'eval_loss': 0.1314130276441574, 'eval_accuracy': 0.9669811320754716, 'eval_f1': 0.9668743167574876, 'eval_runtime': 0.363, 'eval_samples_per_second': 583.948, 'eval_steps_per_second': 74.371, 'epoch': 10.0}


In [ ]:
https://github.com/MLforHealth/MIMIC_Extract.gi